## Toxic: PCA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [2]:
df = pd.read_pickle('../data/toxictrain.pkl')
print(df.shape)
df.head()

(159571, 24)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,category,rating,...,sent_token,polarity_sentence,polarity_comment,polarity_comment_s,word_count,char_count,char_count_s,polarity_min,polarity_max,polarity_mean
idx,,,,,,,,,,,,,,,,,,,,,
3a4c7758fad18de3,3a4c7758fad18de3,", I hope your retarded kids get anal raped and...",1,1,1,1,1,1,ToSTObThInIH,6,...,"[, I hope your retarded kids get anal raped an...","(-0.6, 1.0, -0.018750000000000003, [-0.4, -0.4...",0.00,0.00,94,494,494,-0.6000,1.0000,-0.018750
24d2b50726b67167,24d2b50726b67167,I am going to murder ZimZalaBim ST47 for being...,1,1,1,1,1,1,ToSTObThInIH,6,...,[I am going to murder ZimZalaBim ST47 for bein...,"(-1.0, -1.0, -1.0, [-1.0])",0.00,0.00,12,68,68,-1.0000,-1.0000,-1.000000
c586b7a2fd575b13,c586b7a2fd575b13,"Shut up you asswipe, we don't care. I'll decap...",1,1,1,1,1,1,ToSTObThInIH,6,...,"[Shut up you asswipe, we don't care., I'll dec...","(-0.2, 0.0, -0.07555555555555557, [0.0, -0.177...",-0.25,-0.25,24,3890,1000,-0.2000,0.0000,-0.075556
77d84b1321c22d9a,77d84b1321c22d9a,"LGBT \n\nyou little fuck , are you a fag , tha...",1,1,1,1,1,1,ToSTObThInIH,6,...,"[LGBT \n\nyou little fuck , are you a fag , th...","(-0.17750000000000005, 0.0, -0.088750000000000...",0.00,0.00,56,280,280,-0.1775,0.0000,-0.088750
1368c10281978876,1368c10281978876,"You're a stupid cunt \n\nFuck you dumb arse, y...",1,1,1,1,1,1,ToSTObThInIH,6,...,"[You're a stupid cunt \n\nFuck you dumb arse, ...","(-0.41250000000000003, -0.41250000000000003, -...",0.00,0.00,59,278,278,-0.4125,-0.4125,-0.412500


### MiniBatchSparsePCA

In [68]:
# https://github.com/lambdaofgod/stackexchange/blob/master/stackoverflow/SPCA%20Word%20Clusters.ipynb
import numpy as np
from sklearn.decomposition import MiniBatchSparsePCA
from sklearn.feature_extraction.text import TfidfVectorizer

In [73]:
def token_topics(data, cat_list, n_components):
    token_dict = {}
    for item in cat_list:
        data = df[df[item]==1]
        
        vectorizer = TfidfVectorizer(min_df=5, token_pattern='[a-zA-Z]+', stop_words='english')
        X = vectorizer.fit_transform(data)
        print('dictionary has ', len(vectorizer.vocabulary_), 'entries')
        X_dense = X.toarray()

        spca = MiniBatchSparsePCA(n_components=n_components, alpha=0.04,batch_size=30, n_iter=100, random_state=0)

        %time X_train_reduced = spca.fit_transform(X_dense)

        for i in range(n_components):
            print('component', i, 'has', sum(spca.components_[i, :] != 0), 'nonzero entries')

        component_idxs = [np.where(spca.components_[i, :])[0] for i in range(n_components)]
        idx_to_words = dict((n, word) for (word, n) in vectorizer.vocabulary_.items())

        for i in range(n_components):
            aa = 'Tokens for', i, 'component:'
            bb = 'idx_to_words[i] for i in component_idxs[i]'
        token_dict[item] = {aa,bb}
    return token_dict

In [74]:
df_pca = df[df['rating']>0]
X = df_pca['comment_text']

class_list = ['clean','toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']

token_topics(df_pca, class_list, 1)

dictionary has  2 entries
CPU times: user 140 ms, sys: 11.8 ms, total: 151 ms
Wall time: 410 ms
component 0 has 2 nonzero entries
dictionary has  2 entries
CPU times: user 132 ms, sys: 5.44 ms, total: 137 ms
Wall time: 441 ms
component 0 has 2 nonzero entries
dictionary has  2 entries
CPU times: user 127 ms, sys: 7.96 ms, total: 135 ms
Wall time: 373 ms
component 0 has 2 nonzero entries
dictionary has  2 entries
CPU times: user 120 ms, sys: 4.84 ms, total: 124 ms
Wall time: 339 ms
component 0 has 2 nonzero entries
dictionary has  2 entries
CPU times: user 134 ms, sys: 3.45 ms, total: 138 ms
Wall time: 414 ms
component 0 has 2 nonzero entries
dictionary has  2 entries
CPU times: user 125 ms, sys: 4.47 ms, total: 129 ms
Wall time: 366 ms
component 0 has 2 nonzero entries
dictionary has  2 entries
CPU times: user 136 ms, sys: 5.89 ms, total: 142 ms
Wall time: 504 ms
component 0 has 2 nonzero entries


{'clean': {('Tokens for', 0, 'component:'),
  'idx_to_words[i] for i in component_idxs[i]'},
 'identity_hate': {('Tokens for', 0, 'component:'),
  'idx_to_words[i] for i in component_idxs[i]'},
 'insult': {('Tokens for', 0, 'component:'),
  'idx_to_words[i] for i in component_idxs[i]'},
 'obscene': {('Tokens for', 0, 'component:'),
  'idx_to_words[i] for i in component_idxs[i]'},
 'severe_toxic': {('Tokens for', 0, 'component:'),
  'idx_to_words[i] for i in component_idxs[i]'},
 'threat': {('Tokens for', 0, 'component:'),
  'idx_to_words[i] for i in component_idxs[i]'},
 'toxic': {('Tokens for', 0, 'component:'),
  'idx_to_words[i] for i in component_idxs[i]'}}

In [62]:
vectorizer = TfidfVectorizer(min_df=5, token_pattern='[a-zA-Z]+', stop_words='english')

X_train = vectorizer.fit_transform(X)

print('dictionary has ', len(vectorizer.vocabulary_), 'entries')

X_train_dense = X_train.toarray()

dictionary has  6673 entries


In [63]:
n_components = 6

spca = MiniBatchSparsePCA(n_components=n_components, alpha=0.04,
                          batch_size=30, n_iter=100, random_state=0)

%time X_train_reduced = spca.fit_transform(X_train_dense)

CPU times: user 19.9 s, sys: 2.39 s, total: 22.3 s
Wall time: 43.3 s


In [64]:
for i in range(n_components):
    print('component', i, 'has', sum(spca.components_[i, :] != 0), 'nonzero entries')

component 0 has 524 nonzero entries
component 1 has 48 nonzero entries
component 2 has 298 nonzero entries
component 3 has 165 nonzero entries
component 4 has 96 nonzero entries
component 5 has 1293 nonzero entries


In [65]:
component_idxs = [np.where(spca.components_[i, :])[0] for i in range(n_components)]

In [66]:
idx_to_words = dict((n, word) for (word, n) in vectorizer.vocabulary_.items())

In [67]:
for i in range(n_components):
    print('Tokens for', i, 'component:')
    print([idx_to_words[i] for i in component_idxs[i]])
    print()

Tokens for 0 component:
['able', 'absolute', 'absolutely', 'accept', 'accuse', 'act', 'actually', 'ad', 'add', 'adding', 'address', 'admin', 'administrators', 'admins', 'agree', 'ahead', 'album', 'allowed', 'angry', 'answer', 'anti', 'anybody', 'anymore', 'apparently', 'arabs', 'aren', 'argue', 'arguing', 'argument', 'article', 'articles', 'ask', 'asking', 'assume', 'attack', 'attitude', 'away', 'bad', 'band', 'banning', 'basis', 'believe', 'best', 'bet', 'better', 'bible', 'big', 'bigger', 'bit', 'blind', 'blocked', 'blood', 'book', 'born', 'bother', 'bothered', 'brain', 'british', 'bully', 'bullying', 'bunch', 'c', 'came', 'care', 'case', 'catch', 'caught', 'censor', 'censorship', 'certain', 'certainly', 'change', 'characters', 'chinese', 'christ', 'cite', 'city', 'claim', 'clear', 'clearly', 'close', 'com', 'come', 'common', 'community', 'completely', 'concerned', 'contact', 'content', 'continue', 'control', 'controversial', 'cool', 'correct', 'couldn', 'count', 'country', 'course',